# Module & Setting

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from function import fx

In [2]:
# 출력 옵션 변경
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows', 100)

In [193]:
# Water Quality Data
w_08 = pd.read_excel('../data/refined/water/water_2008.xlsx', index_col=0)
w_09 = pd.read_excel('../data/refined/water/water_2009.xlsx', index_col=0)
w_10 = pd.read_excel('../data/refined/water/water_2010.xlsx', index_col=0)
w_11 = pd.read_excel('../data/refined/water/water_2011.xlsx', index_col=0)
w_12 = pd.read_excel('../data/refined/water/water_2012.xlsx', index_col=0)
w_13 = pd.read_excel('../data/refined/water/water_2013.xlsx', index_col=0)
w_14 = pd.read_excel('../data/refined/water/water_2014.xlsx', index_col=0)
w_15 = pd.read_excel('../data/refined/water/water_2015.xlsx', index_col=0)
w_16 = pd.read_excel('../data/refined/water/water_2016.xlsx', index_col=0)
w_17 = pd.read_excel('../data/refined/water/water_2017.xlsx', index_col=0)
w_18 = pd.read_excel('../data/refined/water/water_2018.xlsx', index_col=0)
w_19 = pd.read_excel('../data/refined/water/water_2019.xlsx', index_col=0)

In [94]:
# Health Quality Data
h_08 = pd.read_excel('../data/refined/health/health_2008.xlsx', index_col=0)
h_09 = pd.read_excel('../data/refined/health/health_2009.xlsx', index_col=0)
h_10 = pd.read_excel('../data/refined/health/health_2010.xlsx', index_col=0)
h_11 = pd.read_excel('../data/refined/health/health_2011.xlsx', index_col=0)
h_12 = pd.read_excel('../data/refined/health/health_2012.xlsx', index_col=0)
h_13 = pd.read_excel('../data/refined/health/health_2013.xlsx', index_col=0)
h_14 = pd.read_excel('../data/refined/health/health_2014.xlsx', index_col=0)
h_15 = pd.read_excel('../data/refined/health/health_2015.xlsx', index_col=0)
h_16 = pd.read_excel('../data/refined/health/health_2016.xlsx', index_col=0)
h_17 = pd.read_excel('../data/refined/health/health_2017.xlsx', index_col=0)
h_18 = pd.read_excel('../data/refined/health/health_2018.xlsx', index_col=0)
h_19 = pd.read_excel('../data/refined/health/health_2019.xlsx', index_col=0)

In [116]:
class fx:
    def waterQualByCity(water, year):
        '''
        water: 전처리와 결측치 처리가 끝난 dataframe. Column index=3에서 '일반세균' 항목으로 시작해야 함
        year: 해당 연도
        각 지역별 1년 평균 물질 농도를 계산하여 dataframe을 반환
        '''

        lst = []  # Dataframe 만들기 위해서 준비
        cities = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도',
                  '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도']
        concs = water.columns[3:]

        for city in cities:
            ct_conc = [year, city]  # 하나의 도시에 대해, 이름과 모든 물질의 농도를 모은 리스트
            for conc in concs:
                df = water[water['지역'].str.contains(city)]
                ct_water = df.copy()

                if conc == '일반세균(기준:100/ 단위:(CFU/mL))':
                    ct_water['월별물질농도(CFU/일)'] = ct_water['시설용량(㎥/일)'] * ct_water[conc] * (10 ** 6)
                    if ct_water['시설용량(㎥/일)'].sum() == 0:
                        t_conc = 0    # 세종시가 없던 연도에는 0으로 채워진다
                    else:
                        t_conc = (ct_water['월별물질농도(CFU/일)'].sum() / ct_water['시설용량(㎥/일)'].sum()) * (
                                    10 ** -6)  # 2018년, xx지역, 일반세균 평균 농도(CFU/mL)

                elif conc == '수소이온농도(기준:5.8 ~ 8.5/ 단위:-)':
                    ct_water[conc] = ct_water[conc].apply(lambda x: 1 / 10 ** x)  # pH를 [H+](단위:mol/L)로 바꾸는 과정
                    ct_water['월별물질농도(mol/일)'] = ct_water['시설용량(㎥/일)'] * ct_water[conc] * 1000
                    if ct_water['시설용량(㎥/일)'].sum() == 0:
                        t_conc = 0
                    else:
                        t_conc = (ct_water['월별물질농도(mol/일)'].sum() / ct_water[
                            '시설용량(㎥/일)'].sum()) * 0.001  # 2018년, xx지역, 수소이온 평균 농도(mol/L)
                        t_conc = -np.log10(t_conc)  # 원래대로 pH로 변환

                else:  # '색도' 포함
                    ct_water['월별물질농도(mg/일)'] = ct_water['시설용량(㎥/일)'] * ct_water[conc] * 1000
                    if ct_water['시설용량(㎥/일)'].sum() == 0:
                        t_conc = 0
                    else:
                        t_conc = (ct_water['월별물질농도(mg/일)'].sum() / ct_water[
                            '시설용량(㎥/일)'].sum()) * 0.001  # 2018년, xx지역, xx 물질 평균 농도(mg/L)

                ct_conc.append(t_conc)
            lst.append(ct_conc)

        result = pd.DataFrame(lst, columns=['연도', '지역'] + list(concs))
        return result


# if __name__ == '__main__':
#     fx.waterQualByCity(water)

## 12년치 데이터를 모은 water.xlsx 파일을 사용해서, 12년 동안 공통적으로 drop 가능한 컬럼을 확인 
이후 다시 Combin_2008_2019_Data 파일로 돌아가서, 각 연도에서 컬럼을 drop한 뒤 개별 excel 파일로 만들기

In [86]:
df = pd.read_excel('../data/refined/water.xlsx', index_col=0)
df.head()

,검사월,수도사업자,시설명,소재지,수원,시설용량(㎥/일),채수년월일,일반세균(기준:100/ 단위:(CFU/mL)),총대장균군(기준:0/ 단위:MPN),대장균/분원성대장균군(기준:0/ 단위:MPN),납(기준:0.01/ 단위:(mg/L)),불소(기준:1.5/ 단위:(mg/L)),비소(기준:0.01/ 단위:(mg/L)),셀레늄(기준:0.01/ 단위:(mg/L)),수은(기준:0.001/ 단위:(mg/L)),시안(기준:0.01/ 단위:(mg/L)),크롬(기준:0.05/ 단위:(mg/L)),암모니아성질소(기준:0.5/ 단위:(mg/L)),질산성질소(기준:10/ 단위:(mg/L)),카드뮴(기준:0.005/ 단위:(mg/L)),붕소(기준:1/ 단위:(mg/L)),페놀(기준:0.005/ 단위:(mg/L)),다이아지논(기준:0.02/ 단위:(mg/L)),파라티온(기준:0.06/ 단위:(mg/L)),"1,2-디브로모-3-클로로프로판(기준:0.003/ 단위:(mg/L))",페니트로티온(기준:0.04/ 단위:(mg/L)),카바릴(기준:0.07/ 단위:(mg/L)),"1,1,1-트리클로로에탄(기준:0.1/ 단위:(mg/L))",테트라클로로에틸렌(기준:0.01/ 단위:(mg/L)),트리클로로에틸렌(기준:0.03/ 단위:(mg/L)),디클로로메탄(기준:0.02/ 단위:(mg/L)),벤젠(기준:0.01/ 단위:(mg/L)),톨루엔(기준:0.7/ 단위:(mg/L)),에틸벤젠(기준:0.3/ 단위:(mg/L)),크실렌(기준:0.5/ 단위:(mg/L)),"1,1-디클로로에틸렌(기준:0.03/ 단위:(mg/L))",사염화탄소(기준:0.002/ 단위:(mg/L)),경도(기준:300/ 단위:(mg/L)),과망간산칼륨소비량(기준:10/ 단위:(mg/L)),냄새(기준:0/ 단위:(mg/L)),맛(기준:0/ 단위:(mg/L)),동(기준:1/ 단위:(mg/L)),색도(기준:5/ 단위:(도)),세제(기준:0.5/ 단위:(mg/L)),수소이온농도(기준:5.8 ~ 8.5/ 단위:-),아연(기준:3/ 단위:(mg/L)),염소이온(기준:250/ 단위:(mg/L)),증발잔류물(기준:500/ 단위:(mg/L)),철(기준:0.3/ 단위:(mg/L)),망간(기준:0.05/ 단위:(mg/L)),탁도(기준:0.5/ 단위:(NTU)),황산이온(기준:200/ 단위:(mg/L)),알루미늄(기준:0.2/ 단위:(mg/L)),총트리할로메탄(기준:0.1/ 단위:(mg/L)),잔류염소(기준:4/ 단위:(mg/L)),클로로포름(기준:0.08/ 단위:(mg/L)),클로랄하이드레이트(기준:0.03/ 단위:(mg/L)),디브로모아세토니트릴(기준:0.1/ 단위:(mg/L)),디클로로아세토니트릴(기준:0.09/ 단위:(mg/L)),트리클로로아세토니트릴(기준:0.004/ 단위:(mg/L)),할로아세틱에시드(기준:0.1/ 단위:(mg/L)),"1,4-다이옥산(기준:0.05/ 단위:(mg/L))",브로모디클로로메탄(기준:0.03/ 단위:(mg/L)),디브로모클로로메탄(기준:0.1/ 단위:(mg/L)),포름알데히드(기준:0.5/ 단위:(mg/L)),브롬산염(기준:0.01/ 단위:(mg/L))
0,2008-01,NaN,반월,경기도 안산시,NaN,NaN,2008-01-31,0,불검출,불검출,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,0.0,적합,적합,0.000,0,0.0,7.2,0.002,9.0,43,0.0,0.0,0.04,10.0,0.02,0.008,0.00,0.004,0.0058,0.0012,0.0031,0.0023,0.0120,NaN,NaN,NaN,NaN,NaN
1,2008-01,NaN,성남,경기도 성남시 수정구,NaN,NaN,2008-02-14,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65,1.0,적합,적합,0.008,0,0.0,7.2,0.003,13.0,146,0.0,0.0,0.05,11.0,0.03,0.028,0.89,0.023,0.0052,0.0000,0.0025,0.0000,0.0074,NaN,NaN,NaN,NaN,NaN
2,2008-01,NaN,시흥,경기도 안산시,NaN,101000.0,2008-01-31,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,0.0,적합,적합,0.000,0,0.0,7.3,0.000,10.0,73,0.0,0.0,0.05,11.0,0.03,0.010,0.00,0.006,0.0062,0.0011,0.0032,0.0024,0.0163,NaN,NaN,NaN,NaN,NaN
3,2008-01,NaN,일산,경기도 고양시,NaN,0.0,2008-02-14,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72,1.6,적합,적합,0.000,0,0.0,7.4,0.000,18.0,139,0.0,0.0,0.08,16.0,0.00,0.029,0.80,0.020,0.0091,0.0000,0.0035,0.0000,0.0074,NaN,NaN,NaN,NaN,NaN
4,2008-01,NaN,지평,경기도 양평군,NaN,NaN,2008-02-12,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64,0.4,적합,적합,0.000,1,0.0,7.2,0.008,10.0,91,0.0,0.0,0.02,14.0,0.00,0.005,0.81,0.002,0.0027,0.0203,0.0011,0.0000,0.0082,NaN,NaN,NaN,NaN,NaN


In [87]:
# dtype 확인 - object: 총대장균, 대장균, 냄새, 맛
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68326 entries, 0 to 68325
Data columns (total 66 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   검사월                                     68326 non-null  object 
 1   수도사업자                                   68318 non-null  object 
 2   시설명                                     68326 non-null  object 
 3   소재지                                     38316 non-null  object 
 4   수원                                      68009 non-null  object 
 5   시설용량(㎥/일)                               68247 non-null  float64
 6   채수년월일                                   68326 non-null  object 
 7   일반세균(기준:100/ 단위:(CFU/mL))               68326 non-null  int64  
 8   총대장균군(기준:0/ 단위:MPN)                     68326 non-null  object 
 9   대장균/분원성대장균군(기준:0/ 단위:MPN)               68285 non-null  object 
 10  납(기준:0.01/ 단위:(mg/L))                   68279 non-null  fl

In [88]:
# 필요없는 컬럼 drop
df.drop(['시설명', '소재지', '수원', '채수년월일'], axis=1, inplace=True)
df

,검사월,수도사업자,시설용량(㎥/일),일반세균(기준:100/ 단위:(CFU/mL)),총대장균군(기준:0/ 단위:MPN),대장균/분원성대장균군(기준:0/ 단위:MPN),납(기준:0.01/ 단위:(mg/L)),불소(기준:1.5/ 단위:(mg/L)),비소(기준:0.01/ 단위:(mg/L)),셀레늄(기준:0.01/ 단위:(mg/L)),수은(기준:0.001/ 단위:(mg/L)),시안(기준:0.01/ 단위:(mg/L)),크롬(기준:0.05/ 단위:(mg/L)),암모니아성질소(기준:0.5/ 단위:(mg/L)),질산성질소(기준:10/ 단위:(mg/L)),카드뮴(기준:0.005/ 단위:(mg/L)),붕소(기준:1/ 단위:(mg/L)),페놀(기준:0.005/ 단위:(mg/L)),다이아지논(기준:0.02/ 단위:(mg/L)),파라티온(기준:0.06/ 단위:(mg/L)),"1,2-디브로모-3-클로로프로판(기준:0.003/ 단위:(mg/L))",페니트로티온(기준:0.04/ 단위:(mg/L)),카바릴(기준:0.07/ 단위:(mg/L)),"1,1,1-트리클로로에탄(기준:0.1/ 단위:(mg/L))",테트라클로로에틸렌(기준:0.01/ 단위:(mg/L)),트리클로로에틸렌(기준:0.03/ 단위:(mg/L)),디클로로메탄(기준:0.02/ 단위:(mg/L)),벤젠(기준:0.01/ 단위:(mg/L)),톨루엔(기준:0.7/ 단위:(mg/L)),에틸벤젠(기준:0.3/ 단위:(mg/L)),크실렌(기준:0.5/ 단위:(mg/L)),"1,1-디클로로에틸렌(기준:0.03/ 단위:(mg/L))",사염화탄소(기준:0.002/ 단위:(mg/L)),경도(기준:300/ 단위:(mg/L)),과망간산칼륨소비량(기준:10/ 단위:(mg/L)),냄새(기준:0/ 단위:(mg/L)),맛(기준:0/ 단위:(mg/L)),동(기준:1/ 단위:(mg/L)),색도(기준:5/ 단위:(도)),세제(기준:0.5/ 단위:(mg/L)),수소이온농도(기준:5.8 ~ 8.5/ 단위:-),아연(기준:3/ 단위:(mg/L)),염소이온(기준:250/ 단위:(mg/L)),증발잔류물(기준:500/ 단위:(mg/L)),철(기준:0.3/ 단위:(mg/L)),망간(기준:0.05/ 단위:(mg/L)),탁도(기준:0.5/ 단위:(NTU)),황산이온(기준:200/ 단위:(mg/L)),알루미늄(기준:0.2/ 단위:(mg/L)),총트리할로메탄(기준:0.1/ 단위:(mg/L)),잔류염소(기준:4/ 단위:(mg/L)),클로로포름(기준:0.08/ 단위:(mg/L)),클로랄하이드레이트(기준:0.03/ 단위:(mg/L)),디브로모아세토니트릴(기준:0.1/ 단위:(mg/L)),디클로로아세토니트릴(기준:0.09/ 단위:(mg/L)),트리클로로아세토니트릴(기준:0.004/ 단위:(mg/L)),할로아세틱에시드(기준:0.1/ 단위:(mg/L)),"1,4-다이옥산(기준:0.05/ 단위:(mg/L))",브로모디클로로메탄(기준:0.03/ 단위:(mg/L)),디브로모클로로메탄(기준:0.1/ 단위:(mg/L)),포름알데히드(기준:0.5/ 단위:(mg/L)),브롬산염(기준:0.01/ 단위:(mg/L))
0,2008-01,NaN,NaN,0,불검출,불검출,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,0.0,적합,적합,0.000,0,0.0,7.2,0.002,9.0,43,0.0,0.0,0.04,10.0,0.02,0.008,0.00,0.004,0.0058,0.0012,0.0031,0.0023,0.0120,NaN,NaN,NaN,NaN,NaN
1,2008-01,NaN,NaN,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65,1.0,적합,적합,0.008,0,0.0,7.2,0.003,13.0,146,0.0,0.0,0.05,11.0,0.03,0.028,0.89,0.023,0.0052,0.0000,0.0025,0.0000,0.0074,NaN,NaN,NaN,NaN,NaN
2,2008-01,NaN,101000.0,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,0.0,적합,적합,0.000,0,0.0,7.3,0.000,10.0,73,0.0,0.0,0.05,11.0,0.03,0.010,0.00,0.006,0.0062,0.0011,0.0032,0.0024,0.0163,NaN,NaN,NaN,NaN,NaN
3,2008-01,NaN,0.0,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.6,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72,1.6,적합,적합,0.000,0,0.0,7.4,0.000,18.0,139,0.0,0.0,0.08,16.0,0.00,0.029,0.80,0.020,0.0091,0.0000,0.0035,0.0000,0.0074,NaN,NaN,NaN,NaN,NaN
4,2008-01,NaN,NaN,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64,0.4,적합,적합,0.000,1,0.0,7.2,0.008,10.0,91,0.0,0.0,0.02,14.0,0.00,0.005,0.81,0.002,0.0027,0.0203,0.0011,0.0000,0.0082,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68321,2019-12,수자원공사,30000.0,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35,2.6,적합,적합,0.000,0,0.0,6.9,0.004,14.7,83,0.0,0.0,0.09,5.0,0.00,0.014,0.69,0.010,0.0016,0.0000,0.0016,0.0000,0.0100,0.000,0.004,0.0,0.0,NaN
68322,2019-12,수자원공사,61900.0,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77,5.2,적합,적합,0.000,0,0.0,7.3,0.000,13.2,261,0.0,0.0,0.10,18.0,0.04,0.055,0.73,0.049,0.0049,0.0000,0.0015,0.0000,0.0410,0.002,0.006,0.0,0.0,NaN
68323,2019-12,수자원공사,100000.0,0,불검출,불검출,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24,2.2,적합,적합,0.000,0,0.0,6.9,0.003,8.1,58,0.0,0.0,0.10,4.0,0.00,0.020,0.70,0.016,0.0030,0.0000,0.0017,0.0000,0.0180,0.000,0.0

In [90]:
# 총대장균, 대장균 컬럼(8~9열)에서 모든 값이 '불검출'이면 컬럼을 삭제하도록 함
df.iloc[:, 4].value_counts()      # 검출: 3건 --> drop
df.iloc[:, 5].value_counts()      # 모두 불검출

# 위 두 컬럼은 제외
df = df.drop(['총대장균군(기준:0/ 단위:MPN)', '대장균/분원성대장균군(기준:0/ 단위:MPN)'], axis=1)
df.head(2)

불검출    68323
검출         3
Name: 총대장균군(기준:0/ 단위:MPN), dtype: int64

In [92]:
# 냄새, 맛 컬럼
df.loc[:, '냄새(기준:0/ 단위:(mg/L))'].value_counts()   
df.loc[:, '맛(기준:0/ 단위:(mg/L))'].value_counts()        # 모두 적합이므로 컬럼 삭제

df = df.drop(['냄새(기준:0/ 단위:(mg/L))', '맛(기준:0/ 단위:(mg/L))'], axis=1)
df.head(2)

적합    68326
Name: 맛(기준:0/ 단위:(mg/L)), dtype: int64

In [12]:
# 탁도 단위(NTU)의 호환성 때문에 컬럼 drop
df.drop('탁도(기준:0.5/ 단위:(NTU))', axis=1, inplace=True)
df.head()

,검사월,수도사업자,시설용량(㎥/일),일반세균(기준:100/ 단위:(CFU/mL)),납(기준:0.01/ 단위:(mg/L)),불소(기준:1.5/ 단위:(mg/L)),비소(기준:0.01/ 단위:(mg/L)),셀레늄(기준:0.01/ 단위:(mg/L)),수은(기준:0.001/ 단위:(mg/L)),시안(기준:0.01/ 단위:(mg/L)),크롬(기준:0.05/ 단위:(mg/L)),암모니아성질소(기준:0.5/ 단위:(mg/L)),질산성질소(기준:10/ 단위:(mg/L)),카드뮴(기준:0.005/ 단위:(mg/L)),붕소(기준:1/ 단위:(mg/L)),페놀(기준:0.005/ 단위:(mg/L)),다이아지논(기준:0.02/ 단위:(mg/L)),파라티온(기준:0.06/ 단위:(mg/L)),"1,2-디브로모-3-클로로프로판(기준:0.003/ 단위:(mg/L))",페니트로티온(기준:0.04/ 단위:(mg/L)),카바릴(기준:0.07/ 단위:(mg/L)),"1,1,1-트리클로로에탄(기준:0.1/ 단위:(mg/L))",테트라클로로에틸렌(기준:0.01/ 단위:(mg/L)),트리클로로에틸렌(기준:0.03/ 단위:(mg/L)),디클로로메탄(기준:0.02/ 단위:(mg/L)),벤젠(기준:0.01/ 단위:(mg/L)),톨루엔(기준:0.7/ 단위:(mg/L)),에틸벤젠(기준:0.3/ 단위:(mg/L)),크실렌(기준:0.5/ 단위:(mg/L)),"1,1-디클로로에틸렌(기준:0.03/ 단위:(mg/L))",사염화탄소(기준:0.002/ 단위:(mg/L)),경도(기준:300/ 단위:(mg/L)),과망간산칼륨소비량(기준:10/ 단위:(mg/L)),동(기준:1/ 단위:(mg/L)),색도(기준:5/ 단위:(도)),세제(기준:0.5/ 단위:(mg/L)),수소이온농도(기준:5.8 ~ 8.5/ 단위:-),아연(기준:3/ 단위:(mg/L)),염소이온(기준:250/ 단위:(mg/L)),증발잔류물(기준:500/ 단위:(mg/L)),철(기준:0.3/ 단위:(mg/L)),망간(기준:0.05/ 단위:(mg/L)),황산이온(기준:200/ 단위:(mg/L)),알루미늄(기준:0.2/ 단위:(mg/L)),총트리할로메탄(기준:0.1/ 단위:(mg/L)),잔류염소(기준:4/ 단위:(mg/L)),클로로포름(기준:0.08/ 단위:(mg/L)),클로랄하이드레이트(기준:0.03/ 단위:(mg/L)),디브로모아세토니트릴(기준:0.1/ 단위:(mg/L)),디클로로아세토니트릴(기준:0.09/ 단위:(mg/L)),트리클로로아세토니트릴(기준:0.004/ 단위:(mg/L)),할로아세틱에시드(기준:0.1/ 단위:(mg/L)),"1,4-다이옥산(기준:0.05/ 단위:(mg/L))",브로모디클로로메탄(기준:0.03/ 단위:(mg/L)),디브로모클로로메탄(기준:0.1/ 단위:(mg/L)),포름알데히드(기준:0.5/ 단위:(mg/L)),브롬산염(기준:0.01/ 단위:(mg/L))
0,2008-01,0,0.0,0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,0.0,0.000,0,0.0,7.2,0.002,9.0,43,0.0,0.0,10.0,0.02,0.008,0.00,0.004,0.0058,0.0012,0.0031,0.0023,0.0120,0.0,0.0,0.0,0.0,0.0
1,2008-01,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65,1.0,0.008,0,0.0,7.2,0.003,13.0,146,0.0,0.0,11.0,0.03,0.028,0.89,0.023,0.0052,0.0000,0.0025,0.0000,0.0074,0.0,0.0,0.0,0.0,0.0
2,2008-01,0,101000.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,0.0,0.000,0,0.0,7.3,0.000,10.0,73,0.0,0.0,11.0,0.03,0.010,0.00,0.006,0.0062,0.0011,0.0032,0.0024,0.0163,0.0,0.0,0.0,0.0,0.0
3,2008-01,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72,1.6,0.000,0,0.0,7.4,0.000,18.0,139,0.0,0.0,16.0,0.00,0.029,0.80,0.020,0.0091,0.0000,0.0035,0.0000,0.0074,0.0,0.0,0.0,0.0,0.0
4,2008-01,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64,0.4,0.000,1,0.0,7.2,0.008,10.0,91,0.0,0.0,14.0,0.00,0.005,0.81,0.002,0.0027,0.0203,0.0011,0.0000,0.0082,0.0,0.0,0.0,0.0,0.0


In [13]:
df.rename({'수도사업자':'지역'}, axis=1, inplace=True)
df.head(2)

,검사월,지역,시설용량(㎥/일),일반세균(기준:100/ 단위:(CFU/mL)),납(기준:0.01/ 단위:(mg/L)),불소(기준:1.5/ 단위:(mg/L)),비소(기준:0.01/ 단위:(mg/L)),셀레늄(기준:0.01/ 단위:(mg/L)),수은(기준:0.001/ 단위:(mg/L)),시안(기준:0.01/ 단위:(mg/L)),크롬(기준:0.05/ 단위:(mg/L)),암모니아성질소(기준:0.5/ 단위:(mg/L)),질산성질소(기준:10/ 단위:(mg/L)),카드뮴(기준:0.005/ 단위:(mg/L)),붕소(기준:1/ 단위:(mg/L)),페놀(기준:0.005/ 단위:(mg/L)),다이아지논(기준:0.02/ 단위:(mg/L)),파라티온(기준:0.06/ 단위:(mg/L)),"1,2-디브로모-3-클로로프로판(기준:0.003/ 단위:(mg/L))",페니트로티온(기준:0.04/ 단위:(mg/L)),카바릴(기준:0.07/ 단위:(mg/L)),"1,1,1-트리클로로에탄(기준:0.1/ 단위:(mg/L))",테트라클로로에틸렌(기준:0.01/ 단위:(mg/L)),트리클로로에틸렌(기준:0.03/ 단위:(mg/L)),디클로로메탄(기준:0.02/ 단위:(mg/L)),벤젠(기준:0.01/ 단위:(mg/L)),톨루엔(기준:0.7/ 단위:(mg/L)),에틸벤젠(기준:0.3/ 단위:(mg/L)),크실렌(기준:0.5/ 단위:(mg/L)),"1,1-디클로로에틸렌(기준:0.03/ 단위:(mg/L))",사염화탄소(기준:0.002/ 단위:(mg/L)),경도(기준:300/ 단위:(mg/L)),과망간산칼륨소비량(기준:10/ 단위:(mg/L)),동(기준:1/ 단위:(mg/L)),색도(기준:5/ 단위:(도)),세제(기준:0.5/ 단위:(mg/L)),수소이온농도(기준:5.8 ~ 8.5/ 단위:-),아연(기준:3/ 단위:(mg/L)),염소이온(기준:250/ 단위:(mg/L)),증발잔류물(기준:500/ 단위:(mg/L)),철(기준:0.3/ 단위:(mg/L)),망간(기준:0.05/ 단위:(mg/L)),황산이온(기준:200/ 단위:(mg/L)),알루미늄(기준:0.2/ 단위:(mg/L)),총트리할로메탄(기준:0.1/ 단위:(mg/L)),잔류염소(기준:4/ 단위:(mg/L)),클로로포름(기준:0.08/ 단위:(mg/L)),클로랄하이드레이트(기준:0.03/ 단위:(mg/L)),디브로모아세토니트릴(기준:0.1/ 단위:(mg/L)),디클로로아세토니트릴(기준:0.09/ 단위:(mg/L)),트리클로로아세토니트릴(기준:0.004/ 단위:(mg/L)),할로아세틱에시드(기준:0.1/ 단위:(mg/L)),"1,4-다이옥산(기준:0.05/ 단위:(mg/L))",브로모디클로로메탄(기준:0.03/ 단위:(mg/L)),디브로모클로로메탄(기준:0.1/ 단위:(mg/L)),포름알데히드(기준:0.5/ 단위:(mg/L)),브롬산염(기준:0.01/ 단위:(mg/L))
0,2008-01,0,0.0,0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52,0.0,0.000,0,0.0,7.2,0.002,9.0,43,0.0,0.0,10.0,0.02,0.008,0.00,0.004,0.0058,0.0012,0.0031,0.0023,0.0120,0.0,0.0,0.0,0.0,0.0
1,2008-01,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65,1.0,0.008,0,0.0,7.2,0.003,13.0,146,0.0,0.0,11.0,0.03,0.028,0.89,0.023,0.0052,0.0000,0.0025,0.0000,0.0074,0.0,0.0,0.0,0.0,0.0


# Substitution of NaN to Zero
**각 연도의 전처리 방법**
* 전체 컬럼값이 0이더라도 drop 하지 않기 --> 연도별 컬럼 개수 유지하기 위해서
* water와 health 데이터 모두 '세종특별자치시'가 포함되지 않는 경우는 양쪽 모두에서 drop
* 각 연도에 대해 fx.waterQualByCity() 함수를 적용한 뒤, 2008~2019 concat  -->  health 데이터와 형태 맞추기

## Water Data

In [334]:
dfs = []   # concat하기 위해서 dataframe 모으는 list
year = 2008

for df in [w_08, w_09, w_10, w_11, w_12, w_13, w_14, w_15, w_16, w_17, w_18, w_19]:
    df1 = df.fillna(0)    # 전처리 방식
    w = fx.waterQualByCity(df1, year)   # 한 해, 17개 도시로 반환되는 dataframe
    dfs.append(w)
    year += 1

In [335]:
# Concat 12 years
df_12 = pd.concat(dfs, axis=0, ignore_index=True, join='outer')    # outer join에 의해 NaN값이 다시 생김
df_12.fillna(0, inplace=True)

In [336]:
# 세종특별자치시 확인
sj = df_12[df_12['지역'] == '세종특별자치시']    # 한 행이 모두 0이면 원래 '세종시'가 없던 연도
sj.iloc[:, 2:].sum(axis=1)        

7      267.726656
24     304.327998
41     274.518805
58     260.171988
75     230.007014
92     249.400676
109    256.165502
126    250.902019
143    206.472095
160      0.000000
177      0.000000
194      0.000000
dtype: float64

> 수질 데이터(df_12)에 세종시 없는 연도: 2017, 2018, 2019년  -->  전처리 방법과 상관 없음

## Health Index Data

In [337]:
# NaN값이 있는지 확인
for df in [h_08, h_09, h_10, h_11, h_12, h_13, h_14, h_15, h_16, h_17, h_18, h_19]:
    print('NaN 총 개수:', df.isnull().sum().sum())    # 각 연도의 dataframe에는 NaN이 없음

NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0
NaN 총 개수: 0


In [340]:
# Concat 12 years - outer join에 의해, 2008년 연간보건기관이용률 NaN값이 생긴다 -> 전처리 필요
h = pd.concat([h_08, h_09, h_10, h_11, h_12, h_13, h_14, h_15, h_16, h_17, h_18, h_19], axis=0, ignore_index=True, join='outer')
h.rename({'시도':'지역'}, axis=1, inplace=True)  # 수질 데이터에 맞춤

# 이후 전처리에도 사용할 수 있도록 이 상태로 저장한 뒤, 결측치 처리
h.to_excel('../data/refined/health/health.xlsx')
print('저장 완료-------------------------------')

h.fillna(0, inplace=True)

저장 완료-------------------------------


In [341]:
# 세종시 정보 확인
h[h['지역'] == '세종특별자치시']   

,지역,삶의질지수_표준화율,양호한주관적건강수준인지율_표준화율,스트레스인지율_표준화율,우울감경험률_표준화율,주관적구강건강이나쁜인구의분율_표준화율,연도,연간보건기관이용률_표준화율
71,세종특별자치시,0.948,44.4,32.3,4.7,38.0,2012,35.5
88,세종특별자치시,0.953,45.5,34.3,4.9,28.5,2013,37.5
105,세종특별자치시,0.937,45.1,28.5,6.9,31.3,2014,38.3
122,세종특별자치시,0.946,43.1,32.5,7.5,30.6,2015,28.8
139,세종특별자치시,0.958,49.2,25.0,4.9,26.6,2016,26.7
156,세종특별자치시,0.959,48.8,29.5,7.7,26.2,2017,27.8
173,세종특별자치시,0.940,43.6,30.1,6.1,24.4,2018,25.3
190,세종특별자치시,0.965,44.7,29.2,4.9,22.9,2019,24.6


> 건강지표 데이터(h)에 세종시 없는 연도: 2008, 2009, 2010, 2011년 <br>
> 따라서 **2008, 2009, 2010, 2011, 2017, 2018, 2019년 세종시 drop** 

## 양측 dataframe에서 세종시 drop

In [342]:
# 수질 데이터
mask = ((df_12['지역'] == '세종특별자치시') & (df_12['연도'].isin([2008, 2009, 2010, 2011, 2017, 2018, 2019]))) # 이 경우 제외해야 함
x = df_12[~mask]

In [343]:
x = x.iloc[:, 2:]
x.shape

(197, 53)

In [344]:
# 건강지표 데이터
mask = ((h['지역'] == '세종특별자치시') & (h['연도'].isin([2008, 2009, 2010, 2011, 2017, 2018, 2019])))
y = h[~mask]
y = y.iloc[:, 1:]
y.shape

(197, 7)

## Machine Learning to Get Score Value

In [345]:
# Standardization
from sklearn.preprocessing import StandardScaler

In [346]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[-0.11107782, -0.14945897, -0.69756321, ..., -1.07022312,
        -0.39287957, -0.17795358],
       [-0.11107782, -0.14945897, -0.31751231, ..., -1.07022312,
        -0.39287957, -0.17795358],
       [-0.11107782, -0.14945897,  0.47008243, ..., -1.07022312,
        -0.39287957, -0.17795358],
       ...,
       [-0.07504496,  0.39701506,  0.16354846, ..., -0.45539285,
        -0.36095693, -0.17795358],
       [-0.11107782, -0.14945897, -0.05095518, ...,  0.73295274,
         0.4219675 , -0.17795358],
       [-0.11107782, -0.14945897, -0.69756321, ..., -1.01494287,
        -0.39287957, -0.17795358]])

In [347]:
# train, test data 나누기
from sklearn.model_selection import train_test_split

In [348]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.15, random_state=5)
x_train.shape, y_test.shape

((167, 53), (30, 7))

In [349]:
# Linear Regression
from sklearn.linear_model import LinearRegression

In [350]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [351]:
# Evaluation
lr.score(x_test, y_test)  

-0.19020075420314012

# Drop NaN in Row

## Water Data

In [352]:
dfs = []   # concat하기 위해서 dataframe 모으는 list
year = 2008

for df in [w_08, w_09, w_10, w_11, w_12, w_13, w_14, w_15, w_16, w_17, w_18, w_19]:
    df1 = df.dropna(axis=0)    # 전처리 방식
    w = fx.waterQualByCity(df1, year)   # 한 해, 17개 도시로 반환되는 dataframe
    dfs.append(w)
    year += 1

In [353]:
# Concat 12 years
df_12 = pd.concat(dfs, axis=0, ignore_index=True, join='outer')    # outer join에 의해 NaN값이 다시 생김
df_12.dropna(axis=0, inplace=True)
df_12.head()

,연도,지역,일반세균(기준:100/ 단위:(CFU/mL)),납(기준:0.01/ 단위:(mg/L)),불소(기준:1.5/ 단위:(mg/L)),비소(기준:0.01/ 단위:(mg/L)),셀레늄(기준:0.01/ 단위:(mg/L)),수은(기준:0.001/ 단위:(mg/L)),시안(기준:0.01/ 단위:(mg/L)),크롬(기준:0.05/ 단위:(mg/L)),암모니아성질소(기준:0.5/ 단위:(mg/L)),질산성질소(기준:10/ 단위:(mg/L)),카드뮴(기준:0.005/ 단위:(mg/L)),붕소(기준:1/ 단위:(mg/L)),페놀(기준:0.005/ 단위:(mg/L)),다이아지논(기준:0.02/ 단위:(mg/L)),파라티온(기준:0.06/ 단위:(mg/L)),"1,2-디브로모-3-클로로프로판(기준:0.003/ 단위:(mg/L))",페니트로티온(기준:0.04/ 단위:(mg/L)),카바릴(기준:0.07/ 단위:(mg/L)),"1,1,1-트리클로로에탄(기준:0.1/ 단위:(mg/L))",테트라클로로에틸렌(기준:0.01/ 단위:(mg/L)),트리클로로에틸렌(기준:0.03/ 단위:(mg/L)),디클로로메탄(기준:0.02/ 단위:(mg/L)),벤젠(기준:0.01/ 단위:(mg/L)),톨루엔(기준:0.7/ 단위:(mg/L)),에틸벤젠(기준:0.3/ 단위:(mg/L)),크실렌(기준:0.5/ 단위:(mg/L)),"1,1-디클로로에틸렌(기준:0.03/ 단위:(mg/L))",사염화탄소(기준:0.002/ 단위:(mg/L)),경도(기준:300/ 단위:(mg/L)),과망간산칼륨소비량(기준:10/ 단위:(mg/L)),동(기준:1/ 단위:(mg/L)),색도(기준:5/ 단위:(도)),세제(기준:0.5/ 단위:(mg/L)),수소이온농도(기준:5.8 ~ 8.5/ 단위:-),아연(기준:3/ 단위:(mg/L)),염소이온(기준:250/ 단위:(mg/L)),증발잔류물(기준:500/ 단위:(mg/L)),철(기준:0.3/ 단위:(mg/L)),망간(기준:0.05/ 단위:(mg/L)),황산이온(기준:200/ 단위:(mg/L)),알루미늄(기준:0.2/ 단위:(mg/L)),총트리할로메탄(기준:0.1/ 단위:(mg/L)),잔류염소(기준:4/ 단위:(mg/L)),클로로포름(기준:0.08/ 단위:(mg/L)),클로랄하이드레이트(기준:0.03/ 단위:(mg/L)),디브로모아세토니트릴(기준:0.1/ 단위:(mg/L)),디클로로아세토니트릴(기준:0.09/ 단위:(mg/L)),트리클로로아세토니트릴(기준:0.004/ 단위:(mg/L)),할로아세틱에시드(기준:0.1/ 단위:(mg/L)),브로모디클로로메탄(기준:0.03/ 단위:(mg/L)),디브로모클로로메탄(기준:0.1/ 단위:(mg/L)),"1,4-다이옥산(기준:0.05/ 단위:(mg/L))",포름알데히드(기준:0.5/ 단위:(mg/L))
102,2014,서울특별시,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.734466,0.0,0.007595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,70.655510,1.372465,0.000828,0.000000,0.0,7.117217,0.001141,17.589013,122.081812,0.000000,0.000059,14.372549,0.037181,0.012292,0.475652,0.008191,0.002398,0.000135,0.001199,0.0,0.007493,0.003385,0.000716,0.000000,0.000000
103,2014,부산광역시,0.0,0.0,0.040048,0.0,0.0,0.0,0.0,0.0,0.0,1.700851,0.0,0.034268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,80.634939,1.129028,0.000000,0.000000,0.0,6.972529,0.001243,27.963016,193.576619,0.000000,0.000000,38.531464,0.000000,0.035679,0.671406,0.019978,0.000103,0.000035,0.000007,0.0,0.004708,0.010954,0.004529,0.000000,0.000000
104,2014,대구광역시,0.0,0.0,0.133838,0.0,0.0,0.0,0.0,0.0,0.0,1.842172,0.0,0.034949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,70.648359,1.811174,0.000000,0.000000,0.0,7.074203,0.001035,29.065530,150.155934,0.000000,0.000000,26.424242,0.010101,0.026902,0.633119,0.015872,0.001902,0.001047,0.001207,0.0,0.010031,0.009161,0.002830,0.001181,0.000000
105,2014,인천광역시,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.813119,0.0,0.009801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000018,0.0,73.090739,1.862883,0.000002,0.094433,0.0,7.421331,0.000009,17.481022,146.236989,0.000016,0.000000,14.895695,0.050936,0.018446,0.800407,0.011883,0.004874,0.000967,0.002375,0.0,0.015435,0.005447,0.000817,0.000000,0.000001
106,2014,광주광역시,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.734127,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,23.908730,1.257540,0.000000,0.926587,0.0,6.808506,0.003091,8.554563,40.775794,0.000000,0.000619,5.119048,0.000000,0.022605,0.616607,0.015897,0.002262,0.000298,0.002344,0.0,0.021175,0.005022,0.000986,0.000000,0.000000


> 모든 컬럼 값이 다 있는 2014년부터 2019년까지 얻어진다.

In [354]:
# 세종특별자치시 확인
sj = df_12[df_12['지역'] == '세종특별자치시']    # 한 행이 모두 0이면 원래 '세종시'가 없던 연도
sj.iloc[:, 2:].sum(axis=1)        

109    256.165502
126    250.902019
143    206.472095
160      0.000000
177      0.000000
194      0.000000
dtype: float64

> 수질 데이터(df_12)에 세종시 없는 연도: 2017, 2018, 2019년  -->  전처리 방법과 상관 없음

## Health Index Data

In [355]:
# 위에서 저장해둔 2008~2018 합친 health data 가져오기
h = pd.read_excel('../data/refined/health/health.xlsx', index_col=0)
h.dropna(axis=0, inplace=True)
h.head()

,지역,삶의질지수_표준화율,양호한주관적건강수준인지율_표준화율,스트레스인지율_표준화율,우울감경험률_표준화율,주관적구강건강이나쁜인구의분율_표준화율,연도,연간보건기관이용률_표준화율
16,서울특별시,0.964,49.6,31.2,7.8,29.7,2009,15.3
17,부산광역시,0.962,48.6,27.0,6.1,35.0,2009,21.6
18,대구광역시,0.962,43.9,29.2,8.2,38.9,2009,15.3
19,인천광역시,0.952,42.1,31.2,9.8,35.7,2009,18.2
20,광주광역시,0.957,50.3,28.4,7.0,32.8,2009,15.2


In [356]:
# 수질 데이터에 맞춰 2014년 이후만 추출
h = h[h['연도'].isin([2014, 2015, 2016, 2017, 2018, 2019])]
h.head()

,지역,삶의질지수_표준화율,양호한주관적건강수준인지율_표준화율,스트레스인지율_표준화율,우울감경험률_표준화율,주관적구강건강이나쁜인구의분율_표준화율,연도,연간보건기관이용률_표준화율
98,서울특별시,0.946,46.2,30.6,8.1,27.1,2014,21.5
99,부산광역시,0.954,42.6,26.9,5.8,32.8,2014,24.1
100,대구광역시,0.951,41.3,26.0,5.4,30.7,2014,24.5
101,인천광역시,0.938,39.8,33.2,8.3,33.5,2014,20.8
102,광주광역시,0.950,42.9,28.1,6.4,32.3,2014,22.8


In [357]:
# 세종시 정보 확인
h[h['지역'] == '세종특별자치시']   

,지역,삶의질지수_표준화율,양호한주관적건강수준인지율_표준화율,스트레스인지율_표준화율,우울감경험률_표준화율,주관적구강건강이나쁜인구의분율_표준화율,연도,연간보건기관이용률_표준화율
105,세종특별자치시,0.937,45.1,28.5,6.9,31.3,2014,38.3
122,세종특별자치시,0.946,43.1,32.5,7.5,30.6,2015,28.8
139,세종특별자치시,0.958,49.2,25.0,4.9,26.6,2016,26.7
156,세종특별자치시,0.959,48.8,29.5,7.7,26.2,2017,27.8
173,세종특별자치시,0.940,43.6,30.1,6.1,24.4,2018,25.3
190,세종특별자치시,0.965,44.7,29.2,4.9,22.9,2019,24.6


> 건강지표 데이터(h)에 세종시 없는 연도: 2008, 2009, 2010, 2011년 <br>
> 따라서 **2008, 2009, 2010, 2011, 2017, 2018, 2019년 세종시 drop** 

## 양측 dataframe에서 세종시 drop

In [358]:
# 수질 데이터
mask = ((df_12['지역'] == '세종특별자치시') & (df_12['연도'].isin([2008, 2009, 2010, 2011, 2017, 2018, 2019]))) # 이 경우 제외해야 함
x = df_12[~mask]

In [359]:
x = x.iloc[:, 2:]
x.shape

(99, 53)

In [360]:
# 건강지표 데이터
mask = ((h['지역'] == '세종특별자치시') & (h['연도'].isin([2008, 2009, 2010, 2011, 2017, 2018, 2019])))
y = h[~mask]
y = y.iloc[:, 1:]
y.shape

(99, 7)

## Machine Learning to Get Score Value

In [361]:
# Standardization
from sklearn.preprocessing import StandardScaler

In [362]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[-0.11310318, -0.19117101, -0.68482336, ..., -0.80107279,
        -0.46422618, -0.25580051],
       [-0.11310318, -0.19117101, -0.11579119, ...,  1.05849971,
        -0.46422618, -0.25580051],
       [-0.11310318, -0.19117101,  1.21686125, ...,  0.23001261,
         3.82599774, -0.25580051],
       ...,
       [-0.07651883, -0.19117101,  0.05913653, ..., -0.60146699,
        -0.41854009, -0.25580051],
       [-0.11310318, -0.19117101,  0.12042486, ...,  0.57679388,
         0.36462093, -0.25580051],
       [-0.11310318, -0.19117101, -0.68482336, ..., -1.07538714,
        -0.46422618, -0.25580051]])

In [363]:
# train, test data 나누기
from sklearn.model_selection import train_test_split

In [364]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.15, random_state=5)
x_train.shape, y_test.shape

((84, 53), (15, 7))

In [365]:
# Linear Regression
from sklearn.linear_model import LinearRegression

In [366]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [367]:
# Evaluation
lr.score(x_test, y_test)  

-2.533119911789055

# Substitution of NaN to Mean of the Group

## Water Data

In [368]:
dfs = []   # concat하기 위해서 dataframe 모으는 list
year = 2008

for df in [w_08, w_09, w_10, w_11, w_12, w_13, w_14, w_15, w_16, w_17, w_18, w_19]:
    df1 = df.groupby('지역').apply(lambda x: x.fillna(x.mean()))    # 전처리 방식
    w = fx.waterQualByCity(df1, year)   # 한 해, 17개 도시로 반환되는 dataframe
    dfs.append(w)
    year += 1

In [383]:
# Concat 12 years
df_12 = pd.concat(dfs, axis=0, ignore_index=True, join='outer')    # outer join에 의해 NaN값이 다시 생김
df_12 = df_12.groupby('지역').apply(lambda x: x.fillna(x.mean()))
df_12.shape

(204, 55)

In [384]:
df_12

연도    지역  일반세균(기준:100/ 단위:(CFU/mL))  납(기준:0.01/ 단위:(mg/L))  \
지역                                                                       
강원도  9    2008   강원도                   0.015702               0.000000   
     26   2009   강원도                   0.005790               0.000000   
     43   2010   강원도                   0.014087               0.000000   
     60   2011   강원도                   0.011496               0.000004   
     77   2012   강원도                   0.330733               0.000001   
...        ...   ...                        ...                    ...   
충청북도 129  2015  충청북도                   0.000396               0.000000   
     146  2016  충청북도                   0.004304               0.000008   
     163  2017  충청북도                   0.005297               0.000019   
     180  2018  충청북도                   0.004941               0.000000   
     197  2019  충청북도                   0.010740               0.000000   

          불소(기준:1.5/ 단위:(mg/L))  비소(기준:0.01/ 단위:(mg/L))  \
지역                                                        
강원도  9                 0.033542                0.000000   
     26                0.059327                0.000000   
     43                0.049080                0.000007   
     60                0.048162                0.000006   
     77                0.053789                0.000008   
...                         ...                     ...   
충청북도 129               0.037553                0.000014   
     146               0.020444                0.000017   
     163               0.019248                0.000014   
     180               0.005781                0.000015   
     197               0.010008                0.000014   

          셀레늄(기준:0.01/ 단위:(mg/L))  수은(기준:0.001/ 단위:(mg/L))  \
지역                                                           
강원도  9                   0.000024                      0.0   
     26                  0.000000                      0.0   
     43                  0.000000                      0.0   
     60                  0.000000                      0.0   
     77                  0.000004                      0.0   
...                           ...                      ...   
충청북도 129                 0.000000                      0.0   
     146                 0.000000                      0.0   
     163                 0.000000                      0.0   
     180                 0.000000                      0.0   
     197                 0.000000                      0.0   

          시안(기준:0.01/ 단위:(mg/L))  크롬(기준:0.05/ 단위:(mg/L))  \
지역                                                         
강원도  9                       0.0                0.000000   
     26                      0.0                0.000000   
     43                      0.0                0.000000   
     60                      0.0                0.000000   
     77                      0.0                0.000015   
...                          ...                     ...   
충청북도 129                     0.0                0.000000   
     146                     0.0                0.000000   
     163                     0.0                0.000000   
     180                     0.0                0.000000   
     197                     0.0                0.000000   

          암모니아성질소(기준:0.5/ 단위:(mg/L))  질산성질소(기준:10/ 단위:(mg/L))  \
지역                                                              
강원도  9                      0.004567                 1.665756   
     26                     0.001475                 1.526900   
     43                     0.000099                 1.602343   
     60                     0.002185                 1.768960   
     77                     0.005236                 1.677797   
...                              ...                      ...   
충청북도 129                    0.000280                 1.649711   
     146                    0.000604                 1.894749   
     163                    0.001901       

In [370]:
# 세종특별자치시 확인
sj = df_12[df_12['지역'] == '세종특별자치시']    # 한 행이 모두 0이면 원래 '세종시'가 없던 연도
sj.iloc[:, 2:].sum(axis=1) 

지역          
세종특별자치시  7      267.731959
         24     304.328037
         41     274.518805
         58     260.171988
         75     230.007014
         92     249.400676
         109    256.165502
         126    250.902019
         143    206.472095
         160      0.000000
         177      0.000000
         194      0.000000
dtype: float64

> 수질 데이터(df_12)에 세종시 없는 연도: 2017, 2018, 2019년  -->  전처리 방법과 상관 없음

## Health Index Data

In [371]:
# 위에서 저장해둔 2008~2018 합친 health data 가져오기
h = pd.read_excel('../data/refined/health/health.xlsx', index_col=0)
h = h.groupby('지역').apply(lambda x: x.fillna(x.mean()))
h.shape

(200, 8)

In [372]:
# 세종시 정보 확인
h[h['지역'] == '세종특별자치시']   

지역  삶의질지수_표준화율  양호한주관적건강수준인지율_표준화율  스트레스인지율_표준화율  \
지역                                                                   
세종특별자치시 71   세종특별자치시       0.948                44.4          32.3   
        88   세종특별자치시       0.953                45.5          34.3   
        105  세종특별자치시       0.937                45.1          28.5   
        122  세종특별자치시       0.946                43.1          32.5   
        139  세종특별자치시       0.958                49.2          25.0   
        156  세종특별자치시       0.959                48.8          29.5   
        173  세종특별자치시       0.940                43.6          30.1   
        190  세종특별자치시       0.965                44.7          29.2   

             우울감경험률_표준화율  주관적구강건강이나쁜인구의분율_표준화율    연도  연간보건기관이용률_표준화율  
지역                                                                    
세종특별자치시 71           4.7                  38.0  2012            35.5  
        88           4.9                  28.5  2013            37.5  
        105          6.9                  31.3  2014            38.3  
        122          7.5                  30.6  2015            28.8  
        139          4.9                  26.6  2016            26.7  
        156          7.7                  26.2  2017            27.8  
        173          6.1                  24.4  2018            25.3  
        190          4.9                  22.9  2019            24.6

> 건강지표 데이터(h)에 세종시 없는 연도: 2008, 2009, 2010, 2011년 <br>
> 따라서 **2008, 2009, 2010, 2011, 2017, 2018, 2019년 세종시 drop** 

## 양측 dataframe에서 세종시 drop

In [385]:
# 수질 데이터
mask = ((df_12['지역'] == '세종특별자치시') & (df_12['연도'].isin([2008, 2009, 2010, 2011, 2017, 2018, 2019]))) # 이 경우 제외해야 함
x = df_12[~mask]

In [387]:
x = x.iloc[:, 2:]
x.shape
x.shape

(197, 51)

In [389]:
# 건강지표 데이터
mask = ((h['지역'] == '세종특별자치시') & (h['연도'].isin([2008, 2009, 2010, 2011, 2017, 2018, 2019])))
y = h[~mask]
y = y.iloc[:, 1:]
y.shape

(197, 7)

## Machine Learning to Get Score Value

In [376]:
# Standardization
from sklearn.preprocessing import StandardScaler

In [390]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[-0.31486399, -0.31096806,  0.41360944, ..., -0.79089201,
        -0.41073333, -0.23294911],
       [-0.02067145, -0.31096806, -0.26351409, ..., -0.78088638,
        -0.41073333, -0.23294911],
       [-0.13758033,  0.02252499, -0.26351409, ..., -0.81591376,
        -0.45433784, -0.23294911],
       ...,
       [-0.47795522,  0.30702415, -0.26351409, ..., -0.76674358,
         2.42796547, -0.30185864],
       [-0.63160278,  0.36470342, -0.26351409, ..., -0.78913833,
         2.69690157, -0.30185864],
       [-0.58337884,  0.31526405, -0.26351409, ..., -0.76561467,
         2.74877757, -0.26028305]])

In [391]:
# train, test data 나누기
from sklearn.model_selection import train_test_split

In [392]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.15, random_state=5)
x_train.shape, y_test.shape

((167, 51), (30, 7))

In [393]:
# Linear Regression
from sklearn.linear_model import LinearRegression

In [394]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [395]:
# Evaluation
lr.score(x_test, y_test)  

-0.1437665574285702

# 전처리 방법 결정
* NaN -> 0: -0.19
* NaN drop: -2.53
* **NaN -> mean: -0.14**